In [1]:
import torch

# import qpth.solvers.dynamic.solve as dynamic_solver
from qpth.util import get_sizes, extract_nBatch, expandParam
from qpth.solvers import cvxpy, dynamic
from qpth.qp import QPFunction, QPSolvers

In [2]:
def forward_cvxpy(Q_, p_, G_, h_, A_, b_):
    nBatch = extract_nBatch(Q_, p_, G_, h_, A_, b_)
    Q, _ = expandParam(Q_, nBatch, 3)
    p, _ = expandParam(p_, nBatch, 2)
    G, _ = expandParam(G_, nBatch, 3)
    h, _ = expandParam(h_, nBatch, 2)
    A, _ = expandParam(A_, nBatch, 3)
    b, _ = expandParam(b_, nBatch, 2)

    check_Q_spd = True
    if check_Q_spd:
        for i in range(nBatch):
            e, _ = torch.eig(Q[i])
            if not torch.all(e[:,0] > 0):
                raise RuntimeError('Q is not SPD.')

    _, nineq, nz = G.size()
    neq = A.size(1) if A.nelement() > 0 else 0
    assert(neq > 0 or nineq > 0)

    vals = torch.Tensor(nBatch).type_as(Q)
    zhats = torch.Tensor(nBatch, nz).type_as(Q)
    lams = torch.Tensor(nBatch, nineq).type_as(Q)
    nus = torch.Tensor(nBatch, neq).type_as(Q) \
        if neq > 0 else torch.Tensor()
    slacks = torch.Tensor(nBatch, nineq).type_as(Q)
    for i in range(nBatch):
        Ai, bi = (A[i], b[i]) if neq > 0 else (None, None)
        vals[i], zhati, nui, lami, si = cvxpy.forward_single_np(
            *[x.cpu().numpy() if x is not None else None
            for x in (Q[i], p[i], G[i], h[i], Ai, bi)])
        # if zhati[0] is None:
        #     import IPython, sys; IPython.embed(); sys.exit(-1)
        zhats[i] = torch.Tensor(zhati)
        lams[i] = torch.Tensor(lami)
        slacks[i] = torch.Tensor(si)
        if neq > 0:
            nus[i] = torch.Tensor(nui)

    return zhats, lams, nus, slacks


In [10]:
# Example data from: https://ncss-wpengine.netdna-ssl.com/wp-content/themes/ncss/pdf/Procedures/NCSS/Quadratic_Programming.pdf

p = torch.tensor([1, -2, 4], dtype=torch.float).unsqueeze(0).cuda()
Q = torch.tensor([[2,0,1],[0,4,0],[1,0,6]], dtype=torch.float).unsqueeze(0).cuda()
h = torch.tensor([10, -2, 5, 5, 5, 0, 1, 0], dtype=torch.float).unsqueeze(0).cuda()
G = torch.tensor([
    [3, 4, -2],
    [2, -2, -1],
    [1, 0, 0],
    [0, 1, 0],
    [0, 0, 1],
    [-1, 0, 0],
    [0, -1, 0],
    [0, 0, -1]
], dtype=torch.float).unsqueeze(0).cuda()
b = torch.tensor([5], dtype=torch.float).unsqueeze(0).cuda()
A = torch.tensor([
    [2, 3, 4]
], dtype=torch.float).unsqueeze(0).cuda()

In [4]:
# Random Data

neq = 4
nineq = 5
nz = 3
nBatch = 2

_x = torch.rand(nBatch, nz, nz, dtype=torch.float).cuda()
Q = (_x.bmm(_x.transpose(-1,1)))
p = torch.rand(nBatch, nz, dtype=torch.float).cuda()
h = torch.rand(nBatch, nineq, dtype=torch.float).cuda()
G = torch.rand(nBatch, nineq, nz, dtype=torch.float).cuda()
b = torch.rand(nBatch, neq, dtype=torch.float).cuda()
A = torch.rand(nBatch, neq, nz, dtype=torch.float).cuda()

In [16]:
# OptNet 

f = QPFunction(verbose=-1, maxIter=100, solver=QPSolvers.PDIPM_BATCHED)
%timeit f(Q, p, G, h, A, b)
zhat = f(Q, p, G, h, A, b)

716]], device='cuda:0')
tensor([[3.2789e+00, 1.2345e-09, 4.6053e+00, 3.6105e+00, 4.9895e+00, 3.9474e-01,
         2.3895e+00, 1.0527e-02]], device='cuda:0')
tensor([[0.3947, 1.3895, 0.0105]], device='cuda:0')
tensor([[6.7359e-11, 1.7158e-01, 4.6668e-11, 5.9615e-11, 4.2959e-11, 5.3825e-10,
         8.9550e-11, 3.5765e-06]], device='cuda:0')
tensor([[-1.0716]], device='cuda:0')
tensor([[3.2789e+00, 1.2345e-09, 4.6053e+00, 3.6105e+00, 4.9895e+00, 3.9474e-01,
         2.3895e+00, 1.0527e-02]], device='cuda:0')
tensor([[0.3947, 1.3895, 0.0105]], device='cuda:0')
tensor([[6.7359e-11, 1.7158e-01, 4.6668e-11, 5.9615e-11, 4.2959e-11, 5.3825e-10,
         8.9550e-11, 3.5765e-06]], device='cuda:0')
tensor([[-1.0716]], device='cuda:0')
tensor([[3.2789e+00, 1.2345e-09, 4.6053e+00, 3.6105e+00, 4.9895e+00, 3.9474e-01,
         2.3895e+00, 1.0527e-02]], device='cuda:0')
tensor([[0.3947, 1.3895, 0.0105]], device='cuda:0')
tensor([[6.7359e-11, 1.7158e-01, 4.6668e-11, 5.9615e-11, 4.2959e-11, 5.3825e-10,


In [14]:
# Layer Dynamic

f = QPFunction(verbose=-1, maxIter=40, solver=QPSolvers.DYNAMIC)
%timeit f(Q, p, G, h, A, b)
zhat = f(Q, p, G, h, A, b)

da:0')
tensor([[0.0000, 0.1627, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.1003]],
       device='cuda:0')
tensor([[-1.0629]], device='cuda:0')
tensor([[ 3.3582e+00, -1.7956e-03,  4.6102e+00,  3.6215e+00,  4.9792e+00,
          3.8979e-01,  2.3785e+00,  2.0780e-02]], device='cuda:0')
tensor([[0.3938, 1.3869, 0.0127]], device='cuda:0')
tensor([[0.0000, 0.1695, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0221]],
       device='cuda:0')
tensor([[-1.0696]], device='cuda:0')
tensor([[ 3.2964e+00, -1.0946e-03,  4.6062e+00,  3.6131e+00,  4.9873e+00,
          3.9378e-01,  2.3869e+00,  1.2666e-02]], device='cuda:0')
tensor([[0.3826, 1.3635, 0.0353]], device='cuda:0')
tensor([[0.0000, 0.1506, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.2404]],
       device='cuda:0')
tensor([[-1.0509]], device='cuda:0')
tensor([[ 3.4689e+00, -3.0522e-03,  4.6174e+00,  3.6365e+00,  4.9647e+00,
          3.8265e-01,  2.3635e+00,  3.5324e-02]], device='cuda:0')
tensor([[0.3763, 1.3500, 0.0483]], device='cuda:0')
tensor

In [17]:
# CVXPY

f = QPFunction(verbose=-1, maxIter=20, solver=QPSolvers.CVXPY)
%timeit f(Q, p, G, h, A, b)
zhat = f(Q, p, G, h, A, b)

.0105]], device='cuda:0')
tensor([[ 0.0000e+00,  1.7158e-01,  0.0000e+00,  0.0000e+00, -1.6051e-15,
          1.0195e-21,  8.0256e-16, -1.1282e-21]], device='cuda:0')
tensor([[-1.0716]], device='cuda:0')
tensor([[3.2789e+00, 1.6856e-22, 4.6053e+00, 3.6105e+00, 4.9895e+00, 3.9474e-01,
         2.3895e+00, 1.0526e-02]], device='cuda:0')
[0.39473684 1.38947368 0.01052632]
tensor([[0.3947, 1.3895, 0.0105]], device='cuda:0')
tensor([[ 0.0000e+00,  1.7158e-01,  0.0000e+00,  0.0000e+00, -1.6051e-15,
          1.0195e-21,  8.0256e-16, -1.1282e-21]], device='cuda:0')
tensor([[-1.0716]], device='cuda:0')
tensor([[3.2789e+00, 1.6856e-22, 4.6053e+00, 3.6105e+00, 4.9895e+00, 3.9474e-01,
         2.3895e+00, 1.0526e-02]], device='cuda:0')
[0.39473684 1.38947368 0.01052632]
tensor([[0.3947, 1.3895, 0.0105]], device='cuda:0')
tensor([[ 0.0000e+00,  1.7158e-01,  0.0000e+00,  0.0000e+00, -1.6051e-15,
          1.0195e-21,  8.0256e-16, -1.1282e-21]], device='cuda:0')
tensor([[-1.0716]], device='cuda:0')


In [13]:
# Standard Forward

# %timeit dynamic.forward(Q, p, G, h, A, b, maxIter=100)
zhat, lams, nus, slacks = dynamic.forward(Q, p, G, h, A, b, maxIter=100)

print(zhat)
print(lams)
print(nus)
print(slacks)

tensor([[-5.4062e+15, -6.4525e+14,  3.2375e+15],
        [-8.6517e+02, -6.4365e+02,  1.9715e+03]], device='cuda:0')
tensor([[0.0000e+00, 0.0000e+00, 0.0000e+00, 1.1468e+13, 0.0000e+00],
        [6.5945e+00, 0.0000e+00, 7.4883e+01, 0.0000e+00, 0.0000e+00]],
       device='cuda:0')
tensor([[-1.8995e+15,  1.7331e+15,  1.1393e+15, -1.4028e+15],
        [-1.3153e+03, -1.9290e+02,  1.6116e+03, -8.8133e+02]], device='cuda:0')
tensor([[-1.1443e+15,  2.3243e+15,  4.2349e+15, -3.4497e+14,  3.3386e+14],
        [-2.6685e+00,  1.2827e+02, -2.5753e+01,  4.0653e+02,  1.0937e+03]],
       device='cuda:0')


In [27]:
# CVXPY

# %timeit forward_cvxpy(Q, p, G, h, A, b)
zhat, lams, nus, slacks = forward_cvxpy(Q, p, G, h, A, b)

print(zhat)
print(lams)
print(nus)
print(slacks)

AssertionError: 